In [1]:
#Import Essential Libraries
import pandas as pd
import numpy as np

import random

In [2]:
#Import the clean patient data and ecg_data
df_ECG_data = pd.read_csv(r'ptbxl_database_cleaned.csv', index_col= 'ecg_id')
ECG_array = np.load(r'ECG_signal.npy')



In [3]:
df_ECG_data.shape

(20699, 21)

In [4]:
ECG_array.shape

(20699, 1000, 12)

In [5]:
df_ECG_data.head()

,patient_id,age,sex,height,weight,recording_date,scp_codes,heart_axis,validated_by,second_opinion,...,validated_by_human,electrodes_problems,strat_fold,filename_lr,filename_hr,Age_Group,Height_Group,Weight_Group,heart_rhythm,AFIB_diagnosis
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,166.0,63.0,1984-11-09 09:17:34,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",NaN,NaN,False,...,True,no,3,records100/00000/00001_lr,records500/00000/00001_hr,"(55, 70]","(90, 166]","(60, 70]",SR,SR
2,13243.0,19.0,0,166.0,70.0,1984-11-14 12:55:37,"{'NORM': 80.0, 'SBRAD': 0.0}",NaN,NaN,False,...,True,no,2,records100/00000/00002_lr,records500/00000/00002_hr,"(0, 55]","(90, 166]","(60, 70]",SBRAD,OTHER
3,20372.0,37.0,1,166.0,69.0,1984-11-15 12:49:10,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,False,...,True,no,5,records100/00000/00003_lr,records500/00000/00003_hr,"(0, 55]","(90, 166]","(60, 70]",SR,SR
4,17014.0,24.0,0,166.0,82.0,1984-11-15 13:44:57,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,False,...,True,no,3,records100/00000/00004_lr,records500/00000/00004_hr,"(0, 55]","(90, 166]","(80, 210]",SR,SR
5,17448.0,19.0,1,166.0,70.0,1984-11-17 10:43:15,"{'NORM': 100.0, 'SR': 0.0}",NaN,NaN,False,...,True,no,4,records100/00000/00005_lr,records500/00000/00005_hr,"(0, 55]","(90, 166]","(60, 70]",SR,SR


In [6]:
df_ECG_data.AFIB_diagnosis.value_counts()

SR       16736
OTHER     2473
AFIB      1490
Name: AFIB_diagnosis, dtype: int64

In [7]:
df = df_ECG_data[['strat_fold', 'heart_rhythm', 'AFIB_diagnosis']].reset_index()

In [8]:
df.head()

,ecg_id,strat_fold,heart_rhythm,AFIB_diagnosis
0,1,3,SR,SR
1,2,2,SBRAD,OTHER
2,3,5,SR,SR
3,4,3,SR,SR
4,5,4,SR,SR


In [9]:
y_test = pd.get_dummies(df[df.strat_fold > 8]['AFIB_diagnosis'])
X_test = ECG_array[y_test.index]

In [10]:
y_train = pd.get_dummies(df[df.strat_fold <= 8]['AFIB_diagnosis'])
X_train = ECG_array[y_train.index]

In [12]:
y_test.to_csv('y_test.csv')
y_train.to_csv('y_train.csv')

np.save('X_test.npy', X_test)
np.save('X_train.npy', X_train)